# Embedding Search in Postgres


Start the postgres container, run

```sh
$ make run
```

In [1]:
# Install dependencies for the jupyter notebooks
#
# !pip install sqlalchemy psycopg2 pandas

In [2]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(f'postgresql://postgres@localhost:5432/postgres?sslmode=disable')

def execute(query: str):
    """Help function to execute the raw SQL statement to demonstrate the ML capability"""
    with engine.connect() as conn:
        with conn.begin():
            return conn.execute(query)

# Load Rikai pg extensions
execute("CREATE EXTENSION IF NOT EXISTS plpython3u;")
execute("CREATE EXTENSION IF NOT EXISTS rikai")


## We've prepared a simple computer vision datasets.

In [3]:
pd.read_sql("""SELECT * FROM images""", con=engine)

,image_id,image
0,1,(http://farm2.staticflickr.com/1129/4726871278...
1,2,(http://farm4.staticflickr.com/3726/9457732891...


# Show all the pre-populated models

In [4]:
pd.read_sql("SELECT * FROM ml.models", con=engine)

,name,flavor,model_type,uri,options
0,ssd,pytorch,ssd,None,{}
1,embedding,pytorch,features,None,{}


## Lets make some embeddings using pre-trained ResNet (PyTorch)

In [5]:
execute("ALTER TABLE images ADD COLUMN embedding real[]")
execute("UPDATE images SET embedding = ml.embedding(image);")
pd.read_sql("SELECT * FROM images", con=engine)

,image_id,image,embedding
0,1,(http://farm2.staticflickr.com/1129/4726871278...,"[0.07276332, 0.57534903, 0.40776607, 0.2994129..."
1,2,(http://farm4.staticflickr.com/3726/9457732891...,"[0.26861557, 0.58947176, 0.30805334, 0.5611011..."


In [6]:
# Train a Sklearn PCA model and will use it for dimention reduction.
execute("SELECT ml.train('pca', 'pca', 'images', 'embedding')")
pd.read_sql("SELECT * FROM ml.models", con=engine)

,name,flavor,model_type,uri,options
0,ssd,pytorch,ssd,None,{}
1,embedding,pytorch,features,None,{}
2,pca,sklearn,pca,/tmp/models/pca/ae869207-554e-4d72-ab81-41d391...,None


# We can then use a vector index to index the embeddings. 

For the simplicity, we are using `pgvector` to demonstrate the capabilty.

In [7]:
execute("CREATE EXTENSION IF NOT EXISTS vector")
execute("ALTER TABLE images ADD COLUMN idx vector(2)")

In [8]:
# Build the vector embedding index

execute("CREATE INDEX ON images USING ivfflat (idx vector_l2_ops);")

# Populate the index with embeddinged (after PCA)
execute("UPDATE images SET idx = ml.pca(embedding);")

In [9]:
# Search based on the embedding

pd.read_sql("SELECT image FROM images ORDER BY idx <=> '[1, 2]' limit 1;", con=engine)

,image
0,(http://farm4.staticflickr.com/3726/9457732891...
